In [ ]:
# add jdbc JAR to spark/jars
# pyspark --jars postgresql-42.2.13.jar 

In [1]:
import pyspark.sql.functions as F

In [2]:
spark = SparkSession.builder.appName("postman-pipeline").getOrCreate()

In [3]:
df = spark.read.csv('products.csv', inferSchema=True, header=True)
df.printSchema()
df.count()

root
 |-- name: string (nullable = true)
 |-- sku: string (nullable = true)
 |-- description: string (nullable = true)



861686

In [4]:
df.createOrReplaceTempView("products")

In [5]:
df = spark.sql("SELECT distinct * from products where sku is not null and description is not null")

In [6]:
df.count()

500000

In [7]:
df.show()

+------------------+-------------------+--------------------+
|              name|                sku|         description|
+------------------+-------------------+--------------------+
|    Mary Rodriguez| hand-couple-manage|Senior word socia...|
|    Jose Henderson| together-table-oil|Apply girl treatm...|
|    Karen Villegas|     child-somebody|Every tell serve....|
|      Olivia Lynch|forget-matter-avoid|Perhaps environme...|
|     Whitney Wiley|    side-blue-dream|Quickly short soc...|
|  Brittany Johnson|        east-pretty|Indicate view sim...|
|       Paul Morris|    radio-window-us|Society month sho...|
|   Jason Patterson|   night-art-be-act|Entire around pla...|
|      Kiara Gentry|   compare-politics|Air my kind staff...|
| William Hernandez|    skin-should-old|Stock support nee...|
|      Jason Osborn|      tv-close-next|Talk view rate ki...|
| Meagan Mccullough|  woman-environment|Stay above task. ...|
|   Sabrina Mullins|life-responsibility|Computer relation...|
|       

In [8]:
df.write \
    .format("jdbc") \
    .mode("overwrite") \
    .option("url", "jdbc:postgresql://localhost:5432/ruthwik") \
    .option("dbtable", "postman.products") \
    .option("user", "postgres") \
    .option("password", "ruthwik") \
    .save()

In [9]:
no_of_products = spark.sql("SELECT name, count(name) as no_of_products FROM products group by name having count(name)>1 order by 2 desc")

In [10]:
print(no_of_products.count(), "unique products")

84941 unique products


In [11]:
no_of_products.show()

+-----------------+--------------+
|             name|no_of_products|
+-----------------+--------------+
|    Michael Smith|           247|
|  Michael Johnson|           187|
|     Robert Smith|           167|
|Christopher Smith|           159|
|      David Smith|           158|
| Michael Williams|           157|
|       John Smith|           157|
|      James Smith|           152|
|   Jennifer Smith|           151|
|    Michael Brown|           148|
|    David Johnson|           138|
| Jennifer Johnson|           131|
|     John Johnson|           123|
|    James Johnson|           122|
|    Michael Jones|           113|
|   Michael Miller|           110|
|   David Williams|           108|
|   Robert Johnson|           104|
|      David Brown|           103|
|     Joseph Smith|           103|
+-----------------+--------------+
only showing top 20 rows



In [12]:
no_of_products.write \
    .format("jdbc") \
    .mode("overwrite") \
    .option("url", "jdbc:postgresql://localhost:5432/ruthwik") \
    .option("dbtable", "postman.no_of_products") \
    .option("user", "postgres") \
    .option("password", "ruthwik") \
    .save()

In [13]:
# df1 = spark.sql("SELECT count(sku), sku from products group by sku order by 1 desc").show()
# postgres-query: create table postman.products1(name varchar, sku text primary key, description text);
# this write will fail as there are duplicates in sku column

df_agg = df.groupBy(df['sku']).agg(F.collect_list(F.to_json(F.struct(df['name'], df['description']))).alias('name_description'))
df_agg.printSchema()

root
 |-- sku: string (nullable = true)
 |-- name_description: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [14]:
df_agg.count()

466693

In [15]:
df_agg.show()

+-------------------+--------------------+
|                sku|    name_description|
+-------------------+--------------------+
|           a-effort|[{"name":"Toni Sa...|
|  a-loss-foot-quite|[{"name":"Mark Ga...|
|      a-matter-seem|[{"name":"Matthew...|
|  ability-interview|[{"name":"Jaime M...|
| ability-none-carry|[{"name":"Stephen...|
|       able-between|[{"name":"Morgan ...|
|    able-bring-none|[{"name":"Amanda ...|
|     able-much-seek|[{"name":"Brian M...|
|about-each-as-bring|[{"name":"David M...|
|    accept-congress|[{"name":"Kristi ...|
|accept-enter-across|[{"name":"Sydney ...|
| accept-learn-white|[{"name":"Joseph ...|
| according-painting|[{"name":"Christo...|
|account-reduce-yeah|[{"name":"Christi...|
|across-able-quality|[{"name":"Thomas ...|
|across-finally-food|[{"name":"Heidi H...|
|across-partner-list|[{"name":"Cody To...|
|across-teacher-thus|[{"name":"Matthew...|
|    act-song-before|[{"name":"Angela ...|
|   action-our-group|[{"name":"David L...|
+----------

In [16]:
df_agg.write \
    .format("jdbc") \
    .mode("overwrite") \
    .option("url", "jdbc:postgresql://localhost:5432/ruthwik") \
    .option("dbtable", "postman.products1") \
    .option("user", "postgres") \
    .option("password", "ruthwik") \
    .save()